In [55]:
#!usr/bin/envs python

# The data consists of predator and prey mass by type of feeding interaction
# and predator lifestage. We need to subset the data first by type of
# feeding interation, then by predator lifestage.
# linear models are required for ln(Predmass) ~ ln(Preymass) for predator
# lifestage, for type of feeding interation.


import pdb
from scipy import stats
import pandas as pd
import numpy as np
from pprint import pprint as pp

In [88]:
# convert mg to grams
mydata = pd.read_csv("../Data/EcolArchives-E089-51-D1.csv")

for i in range(1,len(mydata["Prey.mass.unit"])):
    if mydata["Prey.mass.unit"][i] != "g" :
        mydata["Prey.mass"][i] == i * 1/1000
        
print(mydata["Prey.mass"][284])
mydata.loc[mydata['Prey.mass.unit'] == 'mg', 'Prey.mass'] = mydata['Prey.mass'] * 1e-3            
(mydata["Prey.mass.unit"]).replace("mg", "g", inplace = True) 

#mydata["Prey.mass.unit"] = mydata["Prey.mass.unit"].map({"mg" :"g", "g" : "g"}) 
print(mydata["Prey.mass"][284])
print(mydata["Prey.mass.unit"][284])
                


0.137
0.000137
g


In [89]:
# take logs of prey mass and pred mass
mydata['logPreyMass'] = np.log(mydata['Prey.mass'])
mydata['logPredMass'] = np.log(mydata['Predator.mass'])

In [90]:
# Create an array of the type of feeding interactions
Feeding = mydata[u'Type.of.feeding.interaction'].unique()
# Create an array of the predator lifestages
life_stages = mydata[u'Predator.lifestage'].unique()

In [91]:
Feeding


array(['predacious/piscivorous', 'piscivorous', 'planktivorous',
       'predacious', 'insectivorous'], dtype=object)

In [99]:
# Subset the database by each feeding interaction
feeding_dict = {elem: pd.DataFrame for elem in Feeding}

In [100]:
feeding_dict

{'insectivorous': pandas.core.frame.DataFrame,
 'piscivorous': pandas.core.frame.DataFrame,
 'planktivorous': pandas.core.frame.DataFrame,
 'predacious': pandas.core.frame.DataFrame,
 'predacious/piscivorous': pandas.core.frame.DataFrame}

In [103]:
for key in feeding_dict.keys():
    mask = mydata[u'Type.of.feeding.interaction'] == key
    feeding_dict[key] = mydata[mask]

In [104]:
feeding_dict


{'insectivorous':        Record.number In-refID IndividualID                Predator  \
 18586          18587      n/a         7245  Oncorhynchus gorbuscha   
 18587          18588      n/a         7246  Oncorhynchus gorbuscha   
 18588          18589      n/a         7247  Oncorhynchus gorbuscha   
 18589          18590      n/a         7248       Oncorhynchus keta   
 18590          18591      n/a         7249       Oncorhynchus keta   
 18591          18592      n/a         7250       Oncorhynchus keta   
 18607          18608      n/a         7263  Oncorhynchus gorbuscha   
 18610          18611      n/a         7264  Oncorhynchus gorbuscha   
 18624          18625      n/a         7276  Oncorhynchus gorbuscha   
 18645          18646      n/a         7291  Oncorhynchus gorbuscha   
 18648          18649      n/a         7292  Oncorhynchus gorbuscha   
 18650          18651      n/a         7293       Oncorhynchus keta   
 18654          18655      n/a         7294      Oncorhynchu

In [106]:
# Now to access a subset FeedingDict['insectivorous'] for example
# Now I can take each element in Feeding Dict, subset in the same way
# by lifestage, do linear model, export data.

# use this for reporting in order to define column order
columns = ['feeding', 'stage', 'slope', 'intercept', 'r_value', 'p_value', 'std_err']


In [107]:

# create empty df
regress_data = pd.DataFrame()

In [110]:

# populate
for key, value in feeding_dict.items():
    for stage in life_stages:
        y = feeding_dict[key]['logPredMass']
        x = feeding_dict[key]['logPreyMass']
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
        mean = feeding_dict[key]['logPredMass'].mean()
        df = pd.DataFrame(
            {'feeding': key,
             'stage': stage,
             'slope': slope,
             'intercept': intercept,
             'r_value': r_value,
             'p_value': p_value,
             'std_err': std_err},
            index=[0]
        )
        regress_data = regress_data.append(df, ignore_index=True)
        print (key, stage, mean)
        print (slope, intercept, r_value, p_value, std_err)

print (regress_data[columns].head())
pass



predacious/piscivorous adult 6.064037522134584
0.531028610582 4.78378800258 0.584083982434 4.19117303972e-16 0.0585242674151
predacious/piscivorous juvenile 6.064037522134584
0.531028610582 4.78378800258 0.584083982434 4.19117303972e-16 0.0585242674151
predacious/piscivorous larva 6.064037522134584
0.531028610582 4.78378800258 0.584083982434 4.19117303972e-16 0.0585242674151
predacious/piscivorous larva / juvenile 6.064037522134584
0.531028610582 4.78378800258 0.584083982434 4.19117303972e-16 0.0585242674151
predacious/piscivorous postlarva/juvenile 6.064037522134584
0.531028610582 4.78378800258 0.584083982434 4.19117303972e-16 0.0585242674151
predacious/piscivorous postlarva 6.064037522134584
0.531028610582 4.78378800258 0.584083982434 4.19117303972e-16 0.0585242674151
piscivorous adult 7.71223438647506
0.186983761526 7.40641715751 0.196452726588 8.17917186428e-180 0.00647514813727
piscivorous juvenile 7.71223438647506
0.186983761526 7.40641715751 0.196452726588 8.17917186428e-180 0.0